<a href="https://colab.research.google.com/github/YokoyamaLab/PythonBasics/blob/main/day09_07CSVFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day09 資料07 Web上のCSVファイルからの読み込み

例
https://www.data.jma.go.jp/obd/stats/data/mdrr/docs/csv_dl_readme.html

ここにある最低気温のデータを使う。

https://www.data.jma.go.jp/obd/stats/data/mdrr/tem_rct/alltable/mntemsadext00_rct.csv

In [ ]:
# (pythonではなく)wgetコマンドでファイルをダウンロード
! wget https://www.data.jma.go.jp/obd/stats/data/mdrr/tem_rct/alltable/mntemsadext00_rct.csv

In [ ]:
import pandas as pd

df = pd.read_csv('mntemsadext00_rct.csv', encoding="shift-jis") #データによってはshift-jisの代わりにutf-8と入れる必要がある場合もあります

# 沖縄県と青森県のデータのみを利用
df_okinawa_aommori = df[df['都道府県'].isin(['沖縄県','青森県'])].reset_index()
df_okinawa_aommori

In [ ]:
#CSVを見るとわかりますが、なんとフィールド名に(今日とか昨日の)日付が入っていてそれが毎日更新されるという
#ダメ仕様で、それに対応するために今日の日付・昨日の日付を算出します。
import datetime
from dateutil.relativedelta import relativedelta
today = datetime.date.today()

# 表中にある今日の最低気温、今年の最低気温で沖縄と青森を区別できるかチャレンジ
X = df_okinawa_aommori[[str(today.day)+'日の最低気温(℃)','今年の最低気温（℃)（'+str((today+ relativedelta(days=-1)).day)+'日まで）']].values
y = df_okinawa_aommori['都道府県'].map( {'沖縄県': 0, '青森県': 1} ).astype(int).values

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 散布図に分類の様子を描画する
def plot_boundary(model, X, Y, target, xlabel, ylabel):
    cmap_dots = ListedColormap([ "#1f77b4", "#ff7f0e", "#2ca02c"])
    cmap_fills = ListedColormap([ "#c6dcec", "#ffdec2", "#cae7ca"])
    plt.figure(figsize=(5, 5))
    if model:
        XX, YY = np.meshgrid(
            np.linspace(X.min()-1, X.max()+1, 200),
            np.linspace(Y.min()-1, Y.max()+1, 200))
        pred = model.predict(np.c_[XX.ravel(), YY.ravel()]).reshape(XX.shape)
        plt.pcolormesh(XX, YY, pred, cmap=cmap_fills, shading="auto")
        plt.contour(XX, YY, pred, colors="gray")
    plt.scatter(X, Y, c=target, cmap=cmap_dots)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 訓練データ、テストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(X, y)

# ランダムフォレストの学習モデルを作る（訓練データで）
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 正解率を調べる（テストデータで）
pred = model.predict(X_test)
score = accuracy_score(y_test, pred)
print("正解率:", score*100, "%")

# この学習モデルの分類の状態を描画する（全データで）
df = pd.DataFrame(X_test)
print(df)
print(y_test)
plot_boundary(model, df[0], df[1], y_test, "df [0]", "df [1]")